## GETTING STOCK DATA

In [14]:
import yfinance as yf
import mysql.connector
import pandas as pd
import numpy as np

def safe_float(value):
    """Convert a value to float if not NaN, otherwise return None."""
    if pd.isnull(value):
        return None
    return float(value)

def main():
    # Database connection parameters
    server = '192.168.0.17'
    username = 'admin'
    password = 'spotify'
    database = 'Stocks_DB'
    
    # Establish connection to MySQL
    conn = mysql.connector.connect(
        host=server,
        user=username,
        password=password,
        database=database
    )
    cursor = conn.cursor()
    
    # Define the ticker symbol
    ticker_symbol = 'DIS'
    
    # Fetch historical stock data using yfinance
    stock = yf.Ticker(ticker_symbol)
    hist = stock.history(period="max")
    
    # Localize the index to UTC to bypass DST-related issues
    if hist.index.tzinfo is None:
        hist.index = hist.index.tz_localize('UTC', nonexistent='shift_forward', ambiguous='NaT')
    else:
        hist.index = hist.index.tz_convert('UTC')
    
    # Create table name based on ticker symbol (e.g., DIS_data)
    table_name = f"{ticker_symbol}_data"
    
    # SQL to create table if it doesn't exist (note: removed trailing comma)
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        date DATE PRIMARY KEY,
        open FLOAT,
        high FLOAT,
        low FLOAT,
        close FLOAT,
        volume BIGINT
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    
    # SQL to insert data into the table with upsert functionality
    insert_query = f"""
    INSERT INTO {table_name} (date, open, high, low, close, volume)
    VALUES (%s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        open = VALUES(open),
        high = VALUES(high),
        low = VALUES(low),
        close = VALUES(close),
        volume = VALUES(volume)
    """
    
    # Iterate over DataFrame rows and insert into the table
    for index, row in hist.iterrows():
        data_tuple = (
            index.date(), 
            safe_float(row.get('Open')), 
            safe_float(row.get('High')), 
            safe_float(row.get('Low')), 
            safe_float(row.get('Close')), 
            int(row.get('Volume')) if not pd.isnull(row.get('Volume')) else None
        )
        cursor.execute(insert_query, data_tuple)
    
    # Commit changes and close the connection
    conn.commit()
    cursor.close()
    conn.close()
    print("Data imported successfully into the MySQL database.")

if __name__ == "__main__":
    main()

Data imported successfully into the MySQL database.


## Script that inserts News DIRECTLY into database

In [11]:
import requests
import datetime
import time
import pandas as pd
import mysql.connector

# ========== CONFIG ==========
API_KEY = "cv6gs71r01qi7f6qb7kgcv6gs71r01qi7f6qb7l0"
BASE_URL = "https://finnhub.io/api/v1/company-news"

# Replace with any stock ticker
ticker = "TSLA"

# Database Configuration
db_config = {
    'host': '192.168.0.17',  # Change this if needed
    'user': 'admin',
    'password': 'spotify',
    'database': 'Stocks_DB'
}

# Define the start date and end date (today)
start_date = datetime.date(2024, 1, 1)
end_date = datetime.date.today()

all_news = []
current_start = start_date

# Maximum number of retries for a 429 error (rate limit)
MAX_RETRIES = 5

# Establish database connection
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Create table if not exists
create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {ticker}_news (
        news_id BIGINT PRIMARY KEY,
        date_time DATE,
        headline TEXT,
        related VARCHAR(10),
        source_ VARCHAR(255),
        summary TEXT,
        sentiment DOUBLE
    );
"""
cursor.execute(create_table_query)
conn.commit()

while current_start <= end_date:
    current_end = current_start + datetime.timedelta(days=6)
    if current_end > end_date:
        current_end = end_date

    params = {
        "symbol": ticker,
        "from": current_start.strftime("%Y-%m-%d"),
        "to": current_end.strftime("%Y-%m-%d"),
        "token": API_KEY
    }

    print(f"Fetching news for {params['from']} to {params['to']} for {ticker}")

    retries = 0
    success = False
    while not success and retries < MAX_RETRIES:
        response = requests.get(BASE_URL, params=params)
        if response.status_code == 200:
            news_items = response.json()
            if news_items:
                all_news.extend(news_items)
            success = True
        elif response.status_code == 429:
            retries += 1
            wait_time = 2 ** retries
            print(f"Rate limit reached. Retrying in {wait_time} seconds (attempt {retries}/{MAX_RETRIES})")
            time.sleep(wait_time)
        else:
            print(f"Error: {response.status_code} for range {params['from']} to {params['to']}")
            success = True  # Exit retry loop on non-429 errors

    current_start = current_end + datetime.timedelta(days=1)
    time.sleep(1)  # Helps avoid rapid-fire requests

# Convert collected news data to DataFrame
df = pd.DataFrame(all_news)

# Helper function to safely convert Unix timestamp to "YYYY-MM-DD"
def safe_convert(ts):
    try:
        ts_val = int(ts)
        if ts_val <= 0:
            return None
        return datetime.datetime.fromtimestamp(ts_val).date()
    except Exception:
        return None

# If the 'datetime' column exists, convert timestamps
if not df.empty and 'datetime' in df.columns:
    df['datetime'] = df['datetime'].apply(safe_convert)

# Insert data into MySQL database
insert_query = f"""
    INSERT INTO {ticker}_news (news_id, date_time, headline, related, source_, summary)
    VALUES (%s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE 
        date_time = VALUES(date_time),
        headline = VALUES(headline),
        related = VALUES(related),
        source_ = VALUES(source_),
        summary = VALUES(summary);
"""

rows_inserted = 0
for _, row in df.iterrows():
    if pd.isna(row.get('id')) or pd.isna(row.get('datetime')):  # Skip if no ID or Date
        continue
    data = (
        int(row['id']), row['datetime'], row.get('headline', ''),
        row.get('related', ''), row.get('source', ''), row.get('summary', '')
    )
    cursor.execute(insert_query, data)
    rows_inserted += 1

conn.commit()
print(f"Inserted {rows_inserted} news articles into {ticker}_news table.")

cursor.close()
conn.close()

Fetching news for 2024-01-01 to 2024-01-07 for TSLA
Fetching news for 2024-01-08 to 2024-01-14 for TSLA
Fetching news for 2024-01-15 to 2024-01-21 for TSLA
Fetching news for 2024-01-22 to 2024-01-28 for TSLA
Fetching news for 2024-01-29 to 2024-02-04 for TSLA
Fetching news for 2024-02-05 to 2024-02-11 for TSLA
Fetching news for 2024-02-12 to 2024-02-18 for TSLA
Fetching news for 2024-02-19 to 2024-02-25 for TSLA
Fetching news for 2024-02-26 to 2024-03-03 for TSLA
Fetching news for 2024-03-04 to 2024-03-10 for TSLA
Fetching news for 2024-03-11 to 2024-03-17 for TSLA
Fetching news for 2024-03-18 to 2024-03-24 for TSLA
Fetching news for 2024-03-25 to 2024-03-31 for TSLA
Fetching news for 2024-04-01 to 2024-04-07 for TSLA
Fetching news for 2024-04-08 to 2024-04-14 for TSLA
Fetching news for 2024-04-15 to 2024-04-21 for TSLA
Fetching news for 2024-04-22 to 2024-04-28 for TSLA
Fetching news for 2024-04-29 to 2024-05-05 for TSLA
Fetching news for 2024-05-06 to 2024-05-12 for TSLA
Fetching new

## OLD GETTING NEWS WITH FINNHUB API

In [2]:
import requests
import datetime
import time
import pandas as pd

# Finnhub API key and endpoint details
API_KEY = "cv6gs71r01qi7f6qb7kgcv6gs71r01qi7f6qb7l0"
BASE_URL = "https://finnhub.io/api/v1/company-news"
symbol = "AAPL"

# Define the start date (adjust as needed) and end date (today)
start_date = datetime.date(2025, 1, 1)  # Example start date
end_date = datetime.date.today()

all_news = []
current_start = start_date

# Maximum number of retries for a 429 error
MAX_RETRIES = 5

while current_start <= end_date:
    # Define a weekly window: from current_start to current_start + 6 days
    current_end = current_start + datetime.timedelta(days=6)
    if current_end > end_date:
        current_end = end_date

    # Format the dates as YYYY-MM-DD strings
    params = {
        "symbol": symbol,
        "from": current_start.strftime("%Y-%m-%d"),
        "to": current_end.strftime("%Y-%m-%d"),
        "token": API_KEY
    }
    
    print(f"Fetching news for {params['from']} to {params['to']}")
    
    retries = 0
    success = False
    while not success and retries < MAX_RETRIES:
        response = requests.get(BASE_URL, params=params)
        if response.status_code == 200:
            news_items = response.json()
            if news_items:
                all_news.extend(news_items)
            success = True
        elif response.status_code == 429:
            # Rate limit error: wait longer and then retry
            retries += 1
            wait_time = 2 ** retries  # exponential backoff
            print(f"Rate limit reached. Retrying in {wait_time} seconds (attempt {retries}/{MAX_RETRIES})")
            time.sleep(wait_time)
        else:
            print(f"Error: {response.status_code} for range {params['from']} to {params['to']}")
            success = True  # exit retry loop on non-429 errors

    # Move to the next week
    current_start = current_end + datetime.timedelta(days=1)
    # Sleep briefly to help avoid rapid-fire requests
    time.sleep(1)

# Convert the collected news data to a DataFrame
df = pd.DataFrame(all_news)

# Helper function to safely convert Unix timestamp to "YYYY-MM-DD"
def safe_convert(ts):
    try:
        ts_val = int(ts)
        if ts_val <= 0:
            return ""
        return datetime.datetime.fromtimestamp(ts_val).strftime("%Y-%m-%d")
    except Exception as e:
        return ""

# If the 'datetime' column exists, convert the Unix timestamp to "YYYY-MM-DD"
if not df.empty and 'datetime' in df.columns:
    df['datetime'] = df['datetime'].apply(safe_convert)

csv_filename = "AAPL_all_news_weekly.csv"
df.to_csv(csv_filename, index=False)
print(f"All news data saved to {csv_filename}")

Fetching news for 2025-01-01 to 2025-01-07
Fetching news for 2025-01-08 to 2025-01-14
Fetching news for 2025-01-15 to 2025-01-21
Fetching news for 2025-01-22 to 2025-01-28
Fetching news for 2025-01-29 to 2025-02-04
Fetching news for 2025-02-05 to 2025-02-11
Fetching news for 2025-02-12 to 2025-02-18
Fetching news for 2025-02-19 to 2025-02-25
Fetching news for 2025-02-26 to 2025-03-04
Fetching news for 2025-03-05 to 2025-03-09
All news data saved to AAPL_all_news_weekly.csv
